In [1]:
import csv
import pandas as pd

In [2]:
def normalize_within_rows(df, features_to_normalize, min_val=0):
    df_normalized = df.copy()
    
    # Select only the columns to normalize
    features_df = df_normalized[features_to_normalize]
    
    # Min-max normalization: (x - min) / (max - min)
    row_min = min_val
    row_max = features_df.max(axis=1)
    df_normalized[features_to_normalize] = features_df.sub(row_min, axis=0).div(row_max - row_min, axis=0)
        
    # Replace NaN values with 0 (occurs when std=0 or max=min)
    df_normalized[features_to_normalize] = df_normalized[features_to_normalize].fillna(0)
    
    return df_normalized


In [3]:
def normalize_columns(df, features_to_normalize):
    df_normalized = df.copy()
    
    # Min-max normalization: (x - min) / (max - min)
    for feature in features_to_normalize:
        col_min = df[feature].min()
        col_max = df[feature].max()
        df_normalized[feature] = (df[feature] - col_min) / (col_max - col_min)

    # Replace NaN values with 0 (occurs when std=0 or max=min)
    df_normalized[features_to_normalize] = df_normalized[features_to_normalize].fillna(0)
    
    return df_normalized

In [4]:
def multiply_dataframes(df1, df2, features_to_multiply):
    # Verify DataFrames have same shape
    if df1.shape != df2.shape:
        raise ValueError("DataFrames must have the same dimensions")
    
    # Verify specified columns exist in both DataFrames
    for col in features_to_multiply:
        if col not in df1.columns or col not in df2.columns:
            raise ValueError(f"Column '{col}' not found in both DataFrames")
    
    # Create a copy of df1 to store results
    result_df = df1.copy()
    
    # Multiply specified features
    for col in features_to_multiply:
        result_df[col] = df1[col] * df2[col]
    
    return result_df

In [5]:
def normalize_combined(df, features_to_normalize):
    normalized_by_rows = normalize_within_rows(df, features_to_normalize)
    normalized_by_columns = normalize_columns(df, features_to_normalize)
    return multiply_dataframes(normalized_by_rows, normalized_by_columns, features_to_normalize)

In [6]:
# Make monsters.csv

file_names = ['mh2g', 'mh3u', 'mh4u', 'mhrs', 'mhwi', 'mhwilds', 'mhxx']
features_to_normalize = ['fire_weak', 'water_weak', 'thunder_weak', 'ice_weak', 'dragon_weak']
normalized_dfs = pd.DataFrame()

for file_name in file_names:
    df = pd.read_csv(f'../db/monster/{file_name}.csv')
    normalized_df = normalize_combined(df, features_to_normalize).round(2)
    # normalized_df.to_csv(f'{file_name}_nomralized.csv', index=False)
    normalized_dfs = pd.concat([normalized_dfs, normalized_df], axis=0)

# normalized_dfs.to_csv('normalized.csv', index=False)

In [ ]:
# Make monster_fights.csv

fights = pd.read_csv('../db/fights.csv').values.tolist()
monsters_csv = pd.read_csv('../db/monster/monsters.csv')
monsters = monsters_csv.values.tolist()
monsters_columns = list(monsters_csv.columns)
monsters_columns_initiator = ['I_'+mc for mc in monsters_columns]
monsters_columns_opponent = ['O_'+mc for mc in monsters_columns]
csv_columns = []
csv_columns.extend(monsters_columns_initiator[1:])
csv_columns.extend(monsters_columns_opponent[1:])
csv_columns.append("Outcome")
same_monsters = pd.read_csv('../db/monster/same_monster.csv').values.tolist()

monsters_dict = {monster[0]: monster[1:] for monster in monsters}
same_monsters_dict = {monster[0]: monster[1] for monster in same_monsters}

def find_same_monster(monster_name) -> list:
    result = []
    for key, value in same_monsters_dict.items():
        if value == monster_name:
            result.append(key)
    return result

result = []
for fight in fights:
    initiator, opponent, outcome = fight
    initiator_data = []
    opponent_data = []
    
    if initiator in monsters_dict and opponent in monsters_dict:
        initiator_data = monsters_dict[initiator]
        opponent_data = monsters_dict[opponent]
        row_data = []
        row_data.extend(initiator_data)
        row_data.extend(opponent_data)
        row = {column: data for data, column in zip(row_data, csv_columns[:-1])}
        row["Outcome"] = outcome
        result.append(row)
    else:
        same_initiators = [initiator]
        same_opponents = [opponent]
        if initiator not in monsters_dict:
            same_initiators = find_same_monster(initiator)
        if opponent not in monsters_dict:
            same_opponents = find_same_monster(opponent)
        wars = []
        for same_initiator in same_initiators:
            for same_opponent in same_opponents:
                wars.append((same_initiator, same_opponent))
        
        for war in wars:
            initiator_data = monsters_dict[war[0]]
            opponent_data = monsters_dict[war[1]]
            row_data = []
            row_data.extend(initiator_data)
            row_data.extend(opponent_data)
            row = {column: data for data, column in zip(row_data, csv_columns[:-1])}
            row["Outcome"] = outcome
            result.append(row)
            
with open('../db/monster_fights.csv', 'w', newline='',) as file:
    keys = csv_columns
    writer = csv.DictWriter(file, fieldnames=keys)

    writer.writeheader()  # Write header
    writer.writerows(result)  # Write data rows
